<a href="https://colab.research.google.com/github/pramodith/llm_exploration/blob/colab/dynamic_prompt_token_dropping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install transformers
!pip install langchain
!pip install datasets
!pip install huggingface_hub
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import List
import torch
from pprint import pprint
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from datasets import load_dataset
from langchain.callbacks import get_openai_callback

In [5]:
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

(…)ingface.co/gpt2/resolve/main/config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

(…)gingface.co/gpt2/resolve/main/vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

(…)gingface.co/gpt2/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)face.co/gpt2/resolve/main/tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

(…)gpt2/resolve/main/generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
def to_tokens_and_probs(model: AutoModelForCausalLM, tokenizer: AutoTokenizer, input_texts: List[str]):
    """
    This function takes a list of input texts and returns a list of tuples (token, prob) for each token in the input text.
    Reference: https://discuss.huggingface.co/t/announcement-generation-get-probabilities-for-generated-output/30075/17

    Args:
        model (AutoModelForCausalLM): _description_
        tokenizer (AutoTokenizer): _description_
        input_texts (List[str]): _description_

    Returns:
        _type_: _description_
    """
    input_ids = tokenizer(input_texts, padding=True, return_tensors="pt").input_ids
    outputs = model(input_ids)
    probs = torch.softmax(outputs.logits, dim=-1).detach()

    # collect the probability of the generated token -- probability at index 0 corresponds to the token at index 1
    probs = probs[:, :-1, :]
    input_ids = input_ids[:, 1:]
    gen_probs = torch.gather(probs, 2, input_ids[:, :, None]).squeeze(-1)

    batch = []
    for input_sentence, input_probs in zip(input_ids, gen_probs):
        text_sequence = []
        for token_pos, (token, p) in enumerate(zip(input_sentence, input_probs)):
            if token not in tokenizer.all_special_ids:
                text_sequence.append((tokenizer.decode(token), p.item(), token_pos, token.item()))
        batch.append(text_sequence)
    return batch

In [7]:
sample_prompt = ["The capital of France is Paris."]
token_probs = to_tokens_and_probs(model, tokenizer, sample_prompt)


In [8]:
token_probs

[[(' capital', 0.00011510786134749651, 0, 3139),
  (' of', 0.1642124354839325, 1, 286),
  (' France', 0.006539602763950825, 2, 4881),
  (' is', 0.12165054678916931, 3, 318),
  (' Paris', 0.032245226204395294, 4, 6342),
  ('.', 0.26118969917297363, 5, 13)]]

Set your key to openai's API's using colab's secrets features.

In [9]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
GPT_35_TURBO_COST_PER_INP_TOKEN = 0.0010/1000

In [23]:
chat = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [31]:
from pydantic import BaseModel
class ReadingComprehensionPrompt(BaseModel):
    system_message: str = SystemMessagePromptTemplate.from_template(f"You are a very smart student in a reading comprehension class."
        "Your teacher is giving you a reading comprehension test. You are given a passage and a question."
        "You must answer the question based on the passage. Be as concise as possible, correct any typos, you may return the excerpt from the passage containing the answer without phrasing it as a sentence.")
    human_message: str = HumanMessagePromptTemplate.from_template("Passage: {passage}\nQuestion: {question}\nAnswer: ")

rc_prompt = ReadingComprehensionPrompt()

In [32]:
def get_llm_response(prompt: BaseModel, chat: ChatOpenAI, passage: str, question: str):
    chat_message = ChatPromptTemplate.from_messages([prompt.system_message, prompt.human_message])
    completed_prompt = chat_message.format_prompt(passage=passage, question=question).to_messages()
    with get_openai_callback() as cb:
      response = chat(completed_prompt)
    return response, cb

In [33]:
get_llm_response(rc_prompt, chat, "The capital of France is Paris.", "What is the capital of Australia?")

(AIMessage(content='The passage does not provide information about the capital of Australia.'),
 Tokens Used: 115
 	Prompt Tokens: 103
 	Completion Tokens: 12
 Successful Requests: 1
 Total Cost (USD): $0.0001785)

In [46]:
def sample_squad_dataset(num_samples: int = 100):
    dataset = load_dataset("squad", split="validation")
    return dataset.shuffle().select(range(num_samples))

In [38]:
squad_dataset = sample_squad_dataset()

In [19]:
squad_dataset[0]

{'id': '5729f9953f37b31900478620',
 'title': 'Immune_system',
 'context': 'Hormones can act as immunomodulators, altering the sensitivity of the immune system. For example, female sex hormones are known immunostimulators of both adaptive and innate immune responses. Some autoimmune diseases such as lupus erythematosus strike women preferentially, and their onset often coincides with puberty. By contrast, male sex hormones such as testosterone seem to be immunosuppressive. Other hormones appear to regulate the immune system as well, most notably prolactin, growth hormone and vitamin D.',
 'question': 'Female sex hormones are immunostimulators of which immune responses?',
 'answers': {'text': ['adaptive and innate immune responses',
   'both adaptive and innate',
   'adaptive and innate immune responses'],
  'answer_start': [154, 149, 154]}}

In [47]:
import numpy as np
def get_token_dropped_text(doc: str, tokenizer:AutoTokenizer, dropout_percent=0.1) -> str:
    token_probs = to_tokens_and_probs(model, tokenizer, [doc])
    tokens = [token[-1] for token in token_probs[0]]
    num_tokens_to_drop = int(len(tokens)*dropout_percent)
    top_10_percent_tokens = sorted(token_probs[0], key=lambda x: x[1], reverse=True)[:len(tokens) // 10]
    dropped_tokens = [t[0] for t in top_10_percent_tokens]
    tokens_after_deletion = np.delete(tokens, [token[2] for token in top_10_percent_tokens])
    # Remove the top 10% of tokens
    dropped_token_text = tokenizer.decode(tokens_after_deletion)
    return dropped_token_text, dropped_tokens


In [48]:
total_dropped_cost = 0
total_cost = 0
dropped_tokens = []
predicted_answer = []
predicted_answer_for_dropped = []
for i in range(4):
  dropped_context, dropped_context_text = get_token_dropped_text(squad_dataset[i]["context"], tokenizer)
  dropped_question, dropped_question_text = get_token_dropped_text(squad_dataset[i]["question"], tokenizer)
  answer, cb = get_llm_response(rc_prompt, chat, passage = squad_dataset[i]["context"], question = squad_dataset[i]["question"])
  dropped_answer, cb_d = get_llm_response(rc_prompt, chat, passage=dropped_context, question=dropped_question)
  dropped_cost = cb_d.prompt_tokens*GPT_35_TURBO_COST_PER_INP_TOKEN
  cost = cb.prompt_tokens*GPT_35_TURBO_COST_PER_INP_TOKEN
  total_dropped_cost += dropped_cost
  total_cost += cost
  print(f"Actual answer is {squad_dataset[i]['answers']}")
  print(f"Answer for dropped text is {dropped_answer}, cost is {total_dropped_cost}")
  print(f"Answer for original text is {answer}, cost is {total_cost}")
  predicted_answer.append(answer.content)
  predicted_answer_for_dropped.append(dropped_answer.content)
  dropped_tokens.extend(dropped_context_text)
  dropped_tokens.extend(dropped_question_text)

Actual answer is {'text': ['through the wilderness of the Maine district and down the Chaudière River to attack the city of Quebec', 'wilderness of the Maine district and down the Chaudière River', 'the wilderness of the Maine district', 'Maine', 'the wilderness of the Maine district and down the Chaudière River'], 'answer_start': [347, 359, 355, 377, 355]}
Answer for dropped text is content='Yes, Shirley was planning an expedition.', cost is 0.00020899999999999998
Answer for original text is content='Shirley was planning an expedition through the wilderness of the Maine district and down the Chaudière River to attack the city of Quebec.', cost is 0.00022099999999999998
Actual answer is {'text': ['William Farel', 'William Farel', 'William Farel'], 'answer_start': [519, 519, 519]}
Answer for dropped text is content='William Farel', cost is 0.000557
Answer for original text is content='William Farel', cost is 0.000592
Actual answer is {'text': ['Anheuser-Busch InBev', 'Anheuser-Busch InB

In [42]:
from collections import Counter
Counter(dropped_tokens)

Counter({'ing': 1,
         'ne': 2,
         'ac': 1,
         ',': 4,
         ' of': 10,
         ' on': 2,
         ' to': 4,
         ' for': 2,
         'ev': 3,
         'ef': 2,
         're': 4,
         ' as': 2,
         'formation': 1,
         ' needed': 1,
         'en': 1,
         'itation': 1,
         ']': 1,
         'ism': 1,
         '.': 2,
         'ots': 1,
         'u': 1,
         ' and': 1,
         ' Testament': 1,
         'ch': 1,
         'user': 1,
         'Bus': 1,
         'os': 2,
         'B': 1,
         ' own': 1,
         'it': 1,
         ' Company': 1,
         ' Bowl': 4,
         ' game': 1,
         ' year': 1,
         '-': 1,
         'rom': 1,
         'bie': 1,
         'land': 1,
         ' with': 1,
         ' chief': 1,
         ' at': 1,
         ' by': 1,
         'itions': 2})

In [43]:
squad_dataset.to_csv("llm_responses")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

4937

In [44]:
squad_dataset.add_column("predicted_answer",predicted_answer)

ArrowInvalid: ignored

In [45]:
predicted_answer

[AIMessage(content='Shirley was planning an expedition through the wilderness of the Maine district and down the Chaudière River to attack the city of Quebec.'),
 AIMessage(content='William Farel'),
 AIMessage(content='Anheuser-Busch InBev'),
 AIMessage(content='Two of the expeditions were successful.')]